# 대화형 검색 - Conversational Search


# Overview

대화형 검색은 사용자가 자연스럽게 질문하면 인공지능 시스템이 이해하고 관련 정보를 제공하는 방식을 말합니다. 단순히 키워드를 검색하는 것이 아니라, 실제 대화를 통해 정보를 얻습니다. 이는 사용자가 보다 자연스럽게 원하는 정보를 찾을 수 있게 해주며, 키워드를 입력하는 대신 대화를 통해 원하는 내용을 명확히 전달할 수 있습니다. 더불어, 지속적인 대화를 통해 추가적인 정보를 요청하거나 세부 사항을 파악하는 것도 가능합니다.


# 사전 준비

이번 단계를 진행하기 위해서는 [시맨틱 검색 단계](./02.semantic_search.ipynb)를 필수적으로 완료하셔야 합니다. Amazon OpenSearch Service로의 연결은 [시맨틱 검색 단계](./02.semantic_search.ipynb)와 동일하게 수행합니다.


패키지를 설치하고 import 합니다.


In [ ]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3
!pip install -q langchain
!pip install -q langchain_community

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from langchain.vectorstores import OpenSearchVectorSearch
import boto3
import json
import textwrap

CloudFormation에서 필요한 정보를 가져와 OpenSearch 도메인에 연결합니다.


In [ ]:
def get_cfn_outputs(stackname, cfn):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)["Stacks"][0]["Outputs"]:
        outputs[output["OutputKey"]] = output["OutputValue"]
    return outputs

In [ ]:
import boto3, json

# region_name = "us-west-2"
session = boto3.Session()
region_name = session.region_name

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

연결이 잘 되었는지 확인합니다.


In [ ]:
import requests

search_model = {"query": {"match": {"name": "OpenSearch-Cohere"}}, "size": 10}

response = requests.get(
    "https://" + aos_host + "/_plugins/_ml/models/_search", auth=auth, json=search_model
)
model_info = json.loads(response.text)
model_id = model_info["hits"]["hits"][0]["_id"]

index_name = "movie_semantic"

In [ ]:
count = aos_client.count(index=index_name)
print(count)

# 대화형 검색 구현하기


## LangChain을 사용한 대화형 AI 에이전트 구성

LangChain 라이브러리를 사용하여 Anthropic의 Claude 언어 모델을 기반으로 한 대화형 AI 에이전트를 구성합니다. 여기서는 **`ConversationBufferWindowMemory`**를 사용하여 최근 10개의 메시지를 기억하는 대화 기억 장치를 생성합니다.


In [ ]:
from langchain.memory import ConversationBufferWindowMemory

from langchain.chains.question_answering import load_qa_chain
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage

model_kwargs = {  # anthropic
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2048,
    "temperature": 0,
}

llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # 파운데이션 모델 지정
    model_kwargs=model_kwargs,
    region_name=region_name,
)  # Claude 속성 구성

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=10, return_messages=True)

## OpenSearch의 하이브리드 Retriever 객체 생성

OpenSearch 엔진을 사용하여 하이브리드 검색을 수행하는 OpenSearchHybridSearchRetriever를 정의합니다. 이 검색기는 LangChain의 BaseRetriever 클래스를 상속받고 있으며, 주요 기능은 다음과 같습니다. OpenSearchHybridSearchRetriever는 추후 대화형 검색 체인에서 retriever 역할을 담당하게 됩니다.


In [ ]:
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.schema import BaseRetriever
from typing import Any, List
from langchain.schema import Document


class OpenSearchHybridSearchRetriever(BaseRetriever):
    os_client: Any
    index_name: str
    model_id: str
    keyword_weight = 0.3
    semantic_weight = 0.7
    k = 10
    minimum_should_match = 0
    filter = []

    def _reset_search_params(
        self,
    ):

        self.k = 10
        self.minimum_should_match = 0
        self.filter = []
        self.keyword_weight = keyword_weight
        self.semantic_weight = semantic_weight

    def _get_relevant_documents(
        self, query_text: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        query = {
            "size": 10,
            "_source": {"exclude": ["text", "vector_field"]},
            "query": {
                "hybrid": {
                    "queries": [
                        {
                            "multi_match": {
                                "query": query_text,
                                "fields": ["title", "plot", "genre", "main_act", "supp_act"],
                            }
                        },
                        {
                            "neural": {
                                "vector_field": {
                                    "query_text": query_text,
                                    "model_id": model_id,
                                    "k": 30,
                                }
                            }
                        },
                    ]
                }
            },
            "search_pipeline": {
                "description": "Post processor for hybrid search",
                "phase_results_processors": [
                    {
                        "normalization-processor": {
                            "normalization": {"technique": "min_max"},
                            "combination": {
                                "technique": "arithmetic_mean",
                                "parameters": {
                                    "weights": [self.keyword_weight, self.semantic_weight]
                                },
                            },
                        }
                    }
                ],
            },
        }
        res = self.os_client.search(index=index_name, body=query)

        query_result = []

        for hit in res["hits"]["hits"]:
            metadata = {"score": hit["_score"], "id": hit["_id"]}

            content = {
                "제목": hit["_source"]["title"],
                "장르": hit["_source"]["genre"],
                "평점": hit["_source"]["rating"],
                "줄거리": hit["_source"]["plot"],
                "주연": hit["_source"]["main_act"],
                "조연": hit["_source"]["supp_act"],
            }

            doc = Document(page_content=json.dumps(content, ensure_ascii=False), metadata=metadata)
            query_result.append(doc)
        return query_result

## 프롬프트 템플릿 정의


### 영화 추천 기본 프롬프트 정의

이 프롬프트는 목록에서 특정 질문에 대한 답변을 생성하는 프롬프트 템플릿을 정의합니다. 프롬프트 템플릿은 주어진 컨텍스트(영화 목록)와 질문을 입력받아 답변을 생성하는 형식으로 구성되어 있습니다. 답변 생성 시에는 정해진 규칙(예의바른 태도, 목록 형식, 간단한 설명 등)을 따르도록 되어 있습니다.


In [ ]:
from langchain import PromptTemplate

prompt_template = """


Human: Here is the list of recommended movies, inside <movies></movies> XML tags.

<movies>
{context}
</movies>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.
    - Answered in list format
    - Always put a short and concise explanation on why you are recommending this movies.

You are a best movie reviewer in Korea. Please recommend a movies from the list above.

Question:
{question}

If the answer is not in the context, just say "추천해드릴만한 영화가 없습니다."


Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

### Condense 템플릿 정의

이 프롬프트 템플릿은 대화 내역과 후속 질문을 입력받아 독립적인 질문을 생성하는 데 사용됩니다. 템플릿의 구조는 다음과 같습니다:

1. **지시사항(instructions)**: 이 섹션에서는 프롬프트의 목적과 입력 형식에 대한 설명을 제공합니다.
2. **대화 내역(chat_history)**: 여기에는 이전 대화 내용이 포함됩니다. **`{chat_history}`** 부분이 실제 대화 내역으로 대체됩니다.
3. **후속 질문(follow-up-question)**: 이 섹션에는 대화 내역에 대한 후속 질문이 포함됩니다. **`{question}`** 부분이 실제 후속 질문으로 대체됩니다.
4. **독립 질문(standalone question)**: 이 부분에서 모델은 대화 내역과 후속 질문을 요약한 독립적인 질문을 생성해야 합니다.

이 프롬프트 템플릿을 사용하면 LangChain의 **`CONDENSE_QUESTION_PROMPT`**에 대화 내역과 후속 질문을 입력할 수 있습니다. 그러면 모델이 이전 대화 내용과 후속 질문을 종합하여 독립적인 질문을 생성합니다. 이렇게 생성된 질문은 대화 맥락을 유지하면서도 독립적으로 이해할 수 있습니다.


In [ ]:
condense_template = """
Generate one standalone question based on the instructions.

<instrunctions>
- You will be given the following conversation between <chat-history> and </chat-history>
- You will be given the following follow up question between <follow-up-question> and </follow-up-question>
- Standalone question should have summary of the previous questions and answers.
</instructions>

<chat-history>
{chat_history}
</chat-history>

<follow-up-question>
{question}
</follow-up-question>

standalone question:
"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_template)

ConversationalRetrievalChain을 통해 대화형 검색을 위한 Chain을 구성합니다. 문서 검색을 위해 앞서 정의한 OpenSearchHybridSearchRetriever를 retriever로 제공하고, 검색 히스토리가 저장되는 memory 객체를 전달합니다.


In [ ]:
from langchain.chains import ConversationalRetrievalChain

memory.clear()

hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=aos_client, index_name=index_name, model_id=model_id
)

conversation_with_retrieval = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=hybrid_retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT},
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    # verbose=True,
)

# 대화형 검색 테스트

이제 실제로 대화형 검색을 시도해봅시다. 먼저 첫 번째 질문으로 영화를 추천받습니다.


In [ ]:
first_question = "초능력을 가진 영웅들이 지구를 지킨다"
chat_response = conversation_with_retrieval.invoke({"question": first_question})

print(textwrap.fill(chat_response["answer"], 80))

In [ ]:
second_question = "그 영화 평점은?"
chat_response = conversation_with_retrieval.invoke({"question": second_question})

print(textwrap.fill(chat_response["answer"], 80))

In [ ]:
third_question = "비슷한 장르의 다른 영화는?"
chat_response = conversation_with_retrieval.invoke({third_question})

print(textwrap.fill(chat_response["answer"], 80))